<a href="https://colab.research.google.com/github/LuisaPolicarpo/Project3/blob/main/Optimization_Project3_Recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [10]:
#Import table
df_inovmovie = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Projects/Project3_my copy/condition_gi.pickle')
df_inovmovie

,index,primaryTitle,tconst,titleType,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,0,The Shawshank Redemption,tt0111161,movie,1994,Drama,9.273049,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,The Godfather,tt5429154,short,2013,Short,9.162608,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,The Dark Knight,tt14607756,short,2020,Short,8.975596,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,The Lord of the Rings: The Return of the King,tt0167260,movie,2003,Drama,8.965110,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Schindler's List,tt0108052,movie,1993,History,8.952729,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097435,1097435,überRICH,tt6456678,short,2017,Short,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1097436,1097436,Čáp - Moments of Decisions,tt15095312,movie,2016,Documentary,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1097437,1097437,Śakra,tt22488024,movie,2023,Adventure,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1097438,1097438,Šentilj-Spielfeld - Border Crossing That Once Was,tt10192898,movie,2009,Documentary,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#NList of columns to choose from for the model
df_inovmovie.columns

Index(['index', 'primaryTitle', 'tconst', 'titleType', 'startYear', 'genres',
       'wheighted_IMDB', 'Action', 'Adult', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', '\N'],
      dtype='object')

In [14]:
#remove nans from weighted


,index,primaryTitle,tconst,titleType,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,0,The Shawshank Redemption,tt0111161,movie,1994,Drama,9.273049,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,The Godfather,tt5429154,short,2013,Short,9.162608,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,The Dark Knight,tt14607756,short,2020,Short,8.975596,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,The Lord of the Rings: The Return of the King,tt0167260,movie,2003,Drama,8.965110,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,Schindler's List,tt0108052,movie,1993,History,8.952729,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360945,360945,Disaster Movie,tt19119720,short,2008,Short,2.864046,0,0,0,...,0,0,1,1,0,0,0,0,0,0
360946,360946,Justin Bieber: Never Say Never,tt1702443,movie,2011,Music,2.785912,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360947,360947,Radhe,tt10888594,movie,2021,Thriller,2.454999,1,0,0,...,0,0,0,0,0,0,1,0,0,0
360948,360948,Reis,tt5988370,movie,2017,Drama,2.370123,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_inovmovie['startYear'].isna().sum()

0

In [18]:
df_inovmovie['Adventure'].isna().sum()

0

In [19]:
df_inovmovie['wheighted_IMDB'].isna().sum()

736490

In [20]:
df_inovmovie_n = df_inovmovie.dropna(subset=['wheighted_IMDB'])

In [21]:
#KNN model

X = df_inovmovie_n[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X)

In [22]:
name = ['The Shawshank Redemption']

array1, array2 = model.kneighbors(df_inovmovie.loc[df_inovmovie['primaryTitle'].isin(name), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

(array([[0.        , 0.40330899, 0.50174455, 0.78764158, 0.81745646]]),
 array([[ 0,  7, 10, 29, 34]]))

In [23]:
list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(df_inovmovie, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,tconst,titleType,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
0,0,The Shawshank Redemption,tt0111161,movie,1994,Drama,9.273049,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,7,Pulp Fiction,tt0110912,movie,1994,Drama,8.869740,0,0,0,...,0,0,0,0,0,0,0,0,0,0.403309
2,10,Forrest Gump,tt0109830,movie,1994,Romance,8.771305,0,0,0,...,1,0,0,0,0,0,0,0,0,0.501745
3,29,It's a Wonderful Life,tt1591000,short,1994,Short,8.485408,0,0,0,...,0,0,1,0,0,0,0,0,0,0.787642
4,34,Léon: The Professional,tt0110413,movie,1994,Drama,8.455593,1,0,0,...,0,0,0,0,0,0,0,0,0,0.817456


In [26]:
#Remove the movie that is to be recomended from the list
name = ('The Shawshank Redemption')

condition_gi_nn = df_inovmovie[df_inovmovie['primaryTitle'].str.contains(name) == False]
# condition_gi_nn = condition_gi.loc[condition_gi['primaryTitle'] != name]

name1 = ['The Shawshank Redemption']

X1 = df_inovmovie_n[['startYear', 'wheighted_IMDB',
       'Adventure']]

model = NearestNeighbors(n_neighbors=5).fit(X1)

array1, array2 = model.kneighbors(df_inovmovie_n.loc[df_inovmovie_n['primaryTitle'].isin(name1), ['startYear', 'wheighted_IMDB', 'Adventure']])
array1, array2

list_1 = array1.tolist()
list_2 = array2.tolist()

flat_list1 = list(np.concatenate(list_1).flat)
flat_list2 = list(np.concatenate(list_2).flat)

d = {'Distance': flat_list1,'index': flat_list2}
df12 = pd.DataFrame(d)
df12

dfnl_df12 = pd.merge(condition_gi_nn, df12, how='inner', on=["index", "index"])
dfnl_df12.sort_values(by = 'Distance').head(5)

,index,primaryTitle,tconst,titleType,startYear,genres,wheighted_IMDB,Action,Adult,Adventure,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N,Distance
0,7,Pulp Fiction,tt0110912,movie,1994,Drama,8.869740,0,0,0,...,0,0,0,0,0,0,0,0,0,0.403309
1,10,Forrest Gump,tt0109830,movie,1994,Romance,8.771305,0,0,0,...,1,0,0,0,0,0,0,0,0,0.501745
2,29,It's a Wonderful Life,tt1591000,short,1994,Short,8.485408,0,0,0,...,0,0,1,0,0,0,0,0,0,0.787642
3,34,Léon: The Professional,tt0110413,movie,1994,Drama,8.455593,1,0,0,...,0,0,0,0,0,0,0,0,0,0.817456
